In [2]:
import ee
ee.Initialize()
import matplotlib
from matplotlib.pylab import plt
import numpy as np
import pandas as pd
import seaborn as sns
#print(s2c.getInfo())
%matplotlib inline

In [3]:
#Time frame for temporal composites
start = '2018-05-01'
end = '2018-09-30'

In [43]:
#Define features
random_samples = 10
extent = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_boundary')
felled = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_felled_only').map(lambda f : f.set('polytype', 'young'))
young = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_young_trees').map(lambda f : f.set('polytype', 'young'))
mature = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_mature_trees').map(lambda f : f.set('polytype', 'young'))
#randomFc = ee.FeatureCollection.randomPoints(young, random_samples).map(lambda f : f.set('polytype', 'young'))
#randomFelled = ee.FeatureCollection.randomPoints(felled, random_samples).map(lambda f : f.set('polytype', 'felled'))
#randomMature = ee.FeatureCollection.randomPoints(mature, random_samples).map(lambda f : f.set('polytype', 'mature'))
#randomFc = randomFc.merge(randomMature).merge(randomFelled)
randomFc = felled.merge(young).merge(mature)

ee.Number({
  "type": "Invocation", 
  "arguments": {
    "collection": {
      "type": "Invocation", 
      "arguments": {
        "collection2": {
          "type": "Invocation", 
          "arguments": {
            "baseAlgorithm": {
              "body": {
                "type": "Invocation", 
                "arguments": {
                  "object": {
                    "type": "ArgumentRef", 
                    "value": "_MAPPING_VAR_0_0"
                  }, 
                  "key": "polytype", 
                  "value": "young"
                }, 
                "functionName": "Element.set"
              }, 
              "argumentNames": [
                "_MAPPING_VAR_0_0"
              ], 
              "type": "Function"
            }, 
            "collection": {
              "type": "Invocation", 
              "arguments": {
                "tableId": "users/tomw_ee/Inputs/kielder_mature_trees"
              }, 
              "functionName": "Collection.loadTab

In [5]:
#Masking function for S2
def maskS2clouds(image):
    qa = image.select('QA60')
     #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 2**10
    cirrusBitMask = 2**11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    #.and(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask).divide(10000)

In [6]:
#Define image collections
s2c = ee.ImageCollection('COPERNICUS/S2').filterDate(start, end).filterBounds(extent).map(maskS2clouds)
s1c = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(extent).filterDate(start, end).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

### 1. Distribution of pixel values for different land uses
* Reduce image collection to a median composite over the time frame
* Sample the pixel values at all random points for mature, young, felled land uses

In [7]:
#Median composite creation
s1_med = s1c.median()
s2_med = s2c.median()
s2_med= s2_med.addBands(s2_med.normalizedDifference(['B8', 'B4']).rename('ndvi'))

In [8]:
s1_red = s1_med.reduceRegions(randomFc, ee.Reducer.first(),10)
s2_red = s1_med.reduceRegions(randomFc, ee.Reducer.first(),10)

In [9]:
def export_results(coll, output_file_name):
    task=ee.batch.Export.table.toDrive(
            collection=coll,
            folder = 'Pixel_Values',
            fileFormat='CSV',
            fileNamePrefix=output_file_name)
    task.start()

In [10]:
export_results(s1_red, 's1_results_median_luse')
export_results(s2_red, 's2_results_median_luse')

In [ ]:
properties = [i['properties'] for i in test]

## 2. Temporal Data September 2016 to September 2018

In [32]:
start = '2018-09-15'
end = '2018-09-30'

In [33]:
s2c = ee.ImageCollection('COPERNICUS/S2').filterDate(start, end).filterBounds(extent).map(maskS2clouds)
s1c = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(extent).filterDate(start, end).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

In [49]:
def temporal_values(img):
    return img.reduceRegions(randomFc, ee.Reducer.first(),10).map(lambda f : f.set('img_date', img.date().format('dd-MM-yyyy')))
    

In [50]:
temporal_s1 = s1c.map(temporal_values).flatten()
temporal_s2 = s2c.map(temporal_values).flatten()

In [51]:
export_results(temporal_s1,'temporal_test2')
expport_results(temporal_s2, 'temporal_test2')

EEException: Collection query aborted after accumulating over 5000 elements.